# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome()

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [3]:
lastname = driver.find_element_by_name('pht_lnm')
lastname.send_keys('nguyen')

In [4]:
#pht-status
profession = Select(driver.find_element_by_name('pht_status'))
profession.select_by_visible_text('Cosmetologists')

In [5]:
button = driver.find_element_by_xpath('//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [44]:
#//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr[2]/td[1]
spaguys = list(driver.find_elements_by_tag_name('tr').text)
for guy in spaguys[1:11]:
    print(guy.get_attribute('innerHTML'))

	<td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:22%;"><span class="results_text">NGUYEN, TOAN HUU </span><br> <span class="default_text">City:</span> <span class="results_text">SAN ANTONIO</span><br> <span class="default_text">County:</span> <span class="results_text">BEXAR</span><br> <span class="default_text">Zip Code:</span> <span class="results_text">78217</span><br><br><br><span class="default_text"> License #(s):</span> <span class="results_text">780948, 1706491, 1699123</span><br><br><span class="default_text">Complaint #</span> <span class="results_text">COS20180004289      </span></td>	<td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:39%;"><span class="default_text">Date:</span> <span class="results_text">5/30/2018</span><br><br>Respondent is assessed an administrative penalty in the amount of $500.</td>	<td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvet

In [46]:
spaguys = list(driver.find_elements_by_tag_name('tr'))
spaguys

AttributeError: 'list' object has no attribute 'text'

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [7]:
for guy in spaguys[1:11]:
        print(guy.find_elements_by_tag_name('span')[0].text)

NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, TRUNG N
NGUYEN, NGAT THI


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [8]:
#//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr[2]/td[3]
#//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr[3]/td[3]

for guy in spaguys[1:11]:
    print(guy.find_elements_by_tag_name('td')[2].text)

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, an

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [43]:
for guy in spaguys[1:11]:
    for span in guy:
        print(span.find_element_by_class_name('results_text'))

TypeError: 'WebElement' object is not iterable

## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

### Save that to a CSV

- Tip: You'll want to use pandas here

In [ ]:
df.to_csv("spa_violations.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.